In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import f_oneway




# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Covariance=['spherical', 'tied', 'diag', 'full']
Params=['kmeans', 'random']
skf = StratifiedKFold(n_splits=10) 
fold=0

algo = 0
for i in Covariance:
        for j in Params:
                test=False
                for train_index, test_index in skf.split(X, y):
                        fold=fold+1
                
                        X_train, X_test = X[train_index,:], X[test_index,:]
                        y_train, y_test = y[train_index], y[test_index]
                        
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i)
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.05:
    
                                y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                        
                        F1=round(f1_score(y_test,y_pred),3)
                       
                       
                        if test==False :
                                data = np.array([[F1]])
                                test=True
                        else :
                                data = np.append(data,np.array([F1]).reshape(1,1),axis=0)
                if algo == 0  :
                        df=pd.DataFrame(data,columns=['[{},{}]'.format(i,j)])
                        
                else :
                        df1=pd.DataFrame(data,columns=['[{},{}]'.format(i,j)])
                        df=pd.concat([df, df1], axis = 1)
                algo = algo + 1
display(df)
                       


              
                        
             


c:\Users\junda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
c:\Users\junda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
c:\Users\junda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
c:\Users\junda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Window

,"[spherical,kmeans]","[spherical,random]","[tied,kmeans]","[tied,random]","[diag,kmeans]","[diag,random]","[full,kmeans]","[full,random]"
0,0.488,0.488,0.800,0.386,0.889,0.889,0.787,0.873
1,0.756,0.194,0.769,0.558,0.923,0.923,0.762,0.762
2,0.651,0.270,0.939,0.280,0.906,0.906,0.842,0.889
3,0.364,0.364,0.917,0.583,0.863,0.863,0.821,0.836
4,0.500,0.375,1.000,0.542,0.923,0.923,0.857,0.906
5,0.380,0.380,0.936,0.537,0.873,0.873,0.787,0.787
6,0.529,0.529,0.957,0.389,0.980,0.980,0.941,0.980
7,0.240,0.732,0.979,0.176,0.960,0.960,0.941,0.960
8,0.370,0.370,0.111,0.939,0.939,0.939,0.906,0.906
9,0.429,0.375,0.933,0.245,0.923,0.923,0.873,0.923


In [8]:
t,p = f_oneway(df['[spherical,kmeans]'],df['[spherical,random]'],df['[tied,kmeans]'],df['[tied,random]'],df['[diag,kmeans]'],df['[diag,random]'],df['[full,kmeans]'],df['[full,random]'])
print('t statistic: %.3f' % t)
print('p value: %.13f' % p)


t statistic: 23.582
p value: 0.0000000000000
